In [1]:
import pair_strategy
import util
import alphavantage_call
import pandas as pd

API_KEY = "YourKeyHere" # AlphaVantage API Key
position_filename = "pair_positions.csv" # where position data will be stored

In [2]:
### Pairs ###
## LGOV/GOVI, EWW/FLMX, BNO/KSA
pairs = [pair_strategy.PairStrategy(tickers=["BNO", "KSA"], 
                                   methodology=pair_strategy.bollinger_linear, 
                                   args = {"lookback":137, "entry": 1, "exit":0.5}),

        pair_strategy.PairStrategy(tickers=["EWW", "FLMX"],
                                   methodology=pair_strategy.bollinger_linear,
                                   args = {"lookback":7, "entry": 1, "exit":0.5}),
                                   
        pair_strategy.PairStrategy(tickers=["GOVI", "LGOV"],
                                   methodology=pair_strategy.bollinger_linear,
                                   args = {"lookback":41, "entry": 1, "exit":0.5})]

In [11]:
import importlib
importlib.reload(alphavantage_call)

#obtain price timeseries
alphavantage_call.get_timeseries(API_KEY, "TIME_SERIES_DAILY", "EWW")

In [4]:
### PairStrategy is timescale agnostic... in here, we create a daily timeseries dataframe, indexed by ticker ###

timeseries_df = pd.DataFrame()

for pair in pairs:
    for ticker in pair.tickers:
        try:
            new_row = alphavantage_call.get_timeseries(API_KEY, "TIME_SERIES_DAILY", ticker)
            new_row.name = ticker
            timeseries_df = pd.concat([timeseries_df, new_row.to_frame().T])
        except Exception as e:
            print(f"An error occurred: {e}")

timeseries_df.columns = pd.to_datetime(timeseries_df.columns)
print(timeseries_df.to_string())

      1999-11-01  1999-11-02  1999-11-03  1999-11-04  1999-11-05  1999-11-08  1999-11-09  1999-11-10  1999-11-11  1999-11-12  1999-11-15  1999-11-16  1999-11-17  1999-11-18  1999-11-19  1999-11-22  1999-11-23  1999-11-24  1999-11-26  1999-11-29  1999-11-30  1999-12-01  1999-12-02  1999-12-03  1999-12-06  1999-12-07  1999-12-08  1999-12-09  1999-12-10  1999-12-13  1999-12-14  1999-12-15  1999-12-16  1999-12-17  1999-12-20  1999-12-21  1999-12-22  1999-12-23  1999-12-27  1999-12-28  1999-12-29  1999-12-30  1999-12-31  2000-01-03  2000-01-04  2000-01-05  2000-01-06  2000-01-07  2000-01-10  2000-01-11  2000-01-12  2000-01-13  2000-01-14  2000-01-18  2000-01-19  2000-01-20  2000-01-21  2000-01-24  2000-01-25  2000-01-26  2000-01-27  2000-01-28  2000-01-31  2000-02-01  2000-02-02  2000-02-03  2000-02-04  2000-02-07  2000-02-08  2000-02-09  2000-02-10  2000-02-11  2000-02-14  2000-02-15  2000-02-16  2000-02-17  2000-02-18  2000-02-22  2000-02-23  2000-02-24  2000-02-25  2000-02-28  2000-02-29

In [5]:
### In here, we create a similar position dataframe indexed by ticker ###

try:
    positions_df = pd.read_csv(position_filename, index_col=0)
    positions_df.columns = pd.to_datetime(positions_df.columns)
except:
    # On first run, creates a 2Nx1 dataframe of 0s, where N is the number of pairs (i.e. 2N is number)
    # of tickers. The column (date) of this position is the last date in the columns of the timeseries df,
    # The pair stratedt will make a decision for 
    positions_df = pd.DataFrame([[0] for ticker in timeseries_df.index], index=timeseries_df.index, columns=[timeseries_df.columns[-1]])

print(positions_df.to_string())

      2024-09-20
BNO            0
KSA            0
EWW            0
FLMX           0
GOVI           0
LGOV           0


In [7]:
### In here, we obtain new positions for each pair trading strategy ###
import importlib
importlib.reload(pair_strategy)

new_positions_df = pd.DataFrame(columns=[pd.Timestamp.today().normalize()])

print(new_positions_df)

for pair in pairs:
    ticker1 = pair.tickers[0]
    ticker2 = pair.tickers[1]
    #print(ticker1)
    #print(ticker2)
    p1, p2 = pair.compute_positions(timeseries_df.loc[ticker1], timeseries_df.loc[ticker2], positions_df.loc[ticker1].iloc[-1], positions_df.loc[ticker2].iloc[-1])
    new_positions_df.loc[ticker1] = p1
    new_positions_df.loc[ticker2] = p2

positions_df = pd.concat([positions_df, new_positions_df], axis=1)
print(new_positions_df.to_string())

Empty DataFrame
Columns: [2024-09-21 00:00:00]
Index: []
      2024-09-21
BNO     0.000000
KSA     0.000000
EWW     0.381853
FLMX   -1.000000
GOVI    0.813719
LGOV   -1.000000


In [ ]:
### TODO - return Trade objects ###

In [7]:
### Write positions to CSV ###
positions_df.to_csv(position_filename)